# Process stackoverflow data

In [237]:
%load_ext autoreload
%autoreload 2

from google.cloud import bigquery
import ast
import glob
import re
from pathlib import Path

import astor
import pandas as pd
import spacy
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split

from general_utils import apply_parallel, flattenlist
from bs4 import BeautifulSoup

EN = spacy.load('en')
pd.set_option('display.max_colwidth', -1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [238]:
use_cache = False

In [239]:
# TODO: use csv for now, fix credential issue
# bigquery
# https://console.cloud.google.com/bigquery?sq=784649123063:df10117d54ab4ef2a853ce5ba8e583dc

# if use_cache == False: 
#     # get data from bigquery
#     client = bigquery.Client()
#     sql = """
#     SELECT id, body, last_activity_date, score, text as comment
#     FROM `bigquery-public-data.stackoverflow.posts_answers` as answer
#     JOIN (SELECT post_id, text from `bigquery-public-data.stackoverflow.comments`) as c on c.post_id = answer.id
#     WHERE 
#     ( Date(last_activity_date) >= Date("2015-01-01")
#       AND 
#       answer.parent_id IN (
#         SELECT id FROM `bigquery-public-data.stackoverflow.posts_questions`
#         WHERE LOWER(tags) LIKE '%python%'
#       ))

#     """
#     df = client.query(sql).to_dataframe()
#     df.to_csv("data/processed_data/stackoverflow/raw.csv")

# Read from csv files

In [250]:
%%time
# read from csv
df = pd.concat([pd.read_csv(f"data/stackoverflow/processed_data/raw/raw-{i}.csv") for i in range(4)])

CPU times: user 21.4 s, sys: 2.43 s, total: 23.8 s
Wall time: 23.8 s


In [251]:
df.head()

,id,body,last_activity_date,score,comment
0,33734452,"<p>You can call <code>apply</code> and convert your dict values into a set can convert the <code>intersection</code> to a list:</p>\n\n<pre><code>In [15]:\nd={'a': ['j','c'],\n 'b': ['p','r','q'],\n 'c': ['n','k','l','x','a']}\nd\n\nOut[15]:\n{'a': ['j', 'c'], 'b': ['p', 'r', 'q'], 'c': ['n', 'k', 'l', 'x', 'a']}\n\nIn [17]:\ndf['points'] = df.apply(lambda row: list(set(d[row['id']]).intersection(row['points'])), axis=1)\ndf\n\nOut[17]:\n id points\n0 a [c]\n1 b []\n2 c [a, x]\n</code></pre>\n\n<p>As to why you get an error, you're trying to call a <code>.str</code> method on a Series that is a dtype list, they are not strings.</p>",2015-11-16 11:37:21.32 UTC,1,I am getting a key error. Is there any other solution using the two dataframes i added to my post?
1,5099990,"<blockquote>\n <p>One thing I've found really frustrating is that when you trap an import error you're really going down a deep rabbit hole right away.</p>\n</blockquote>\n\n<p>Take the hint.</p>\n\n<p>Do <strong>not</strong> trap import errors ever. The <code>manage.py</code> example is not a best practice.</p>\n\n<p>Do not do that.</p>\n\n<p>import errors should be very, very rare.</p>",2011-02-24 03:15:36.757 UTC,-1,"@S.Lott - I don't mean me as the developer ""know"", I mean in the control I want my code to be able to ""know"" what import failed. Again, defer to Django. That helpful message could not be given, if you relied on Python's built-in exc handling."
2,22650931,"<p>In your code sample, above <code>for rownum in range(1, r+1):</code> place <code>buff = []</code> so that:</p>\n\n<pre><code>buff = []\nfor rownum in range(1, r+1):\n</code></pre>\n\n<p>Then, where you have <code>print(list_v)</code> do the following:</p>\n\n<pre><code>buff.append(list_v)\n</code></pre>\n\n<p>Then your code sample will look like</p>\n\n<pre><code>def make_x_table_right (r,c):\n buff = []\n for rownum in range(1, r+1):\n list_v = []\n for colnum in range(1, c+1):\n v = colnum*rownum\n list_v.append(v)\n buff.append(list_v)\n return buff\n</code></pre>\n\n<p>Make sense? </p>\n\n<p>Also, since your multiplication table is probably going to be square, I'd suggest just passing one parameter <code>n</code> and using <code>n</code> in place of <code>r</code> and <code>c</code>. One other thing, the list comprehension that another user posted as a one-liner comment is perfectly fine, too. Also, it's convention to use variables <code>i</code> and <code>j</code> for matrix loop identifiers. so instead of <code>rownum</code> you'd say <code>i</code> and <code>colnum</code> you'd say <code>j</code>.</p>",2014-03-26 03:28:20.523 UTC,0,"Yup, I thought his shorter method was really cool. But is it possible to actually return the list of lists in a column format? I can generate the horizontal list of lists easy....."
3,49524124,"<p>Only one Change is need. And you can use django-import-export </p>\n\n<p><strong>models.py</strong></p>\n\n<pre><code> class Compositions(models.Model):\n composer_key = models.ForeignKey(\n Composer,\n )\n composition = models.CharField(\n max_length=383,\n unique=False\n )\n date_created = models.DateTimeField(default=timezone.now)\n\n class Meta(object):\n unique_together = (('composer_key','composition'),)\n</code></pre>\n\n<p>override save_instance with try. And ignore error when fail. \n<strong>admin.py</strong></p>\n\n<pre><code> class CompositionsResource(resources.ModelResource):\n\n class Meta:\n model = Compositions\n skip_unchanged = True\n report_skipped = True\n\n def save_instance(self, instance, using_transactions=True, dry_run=False):\n try:\n super(CompositionsResource, self).save_instance(instance, using_transactions, dry_run)\n except IntegrityError:\n pass\n\n class CompositionsAdmin(ImportExportModelAdmin):\n resource_class = CompositionsResource\n\n admin.site.register(Compositions, CompositionsAdmin)\n</code></pre>\n\n<p>and import this</p>\n\n<pre><code>from django.db import Integri

In [252]:
df.shape

(1522068, 5)

In [253]:
def tokenize_text(text):
    "Apply tokenization using spacy to docstrings."
    tokens = EN.tokenizer(text)
    return [token.text.lower() for token in tokens if not token.is_space]


def tokenize_code(text):
    "A very basic procedure for tokenizing code strings."
    return RegexpTokenizer(r'\w+').tokenize(text)


# Aggregate comments

In [254]:
# concat comments for each post

grouped = df.groupby(['id', 'body','last_activity_date','score'],as_index=False).comment.agg(lambda x: "\n".join(x))
deduped_df = pd.DataFrame(grouped)

In [255]:
deduped_df.head(1)

,id,body,last_activity_date,score,comment
0,538,"<p>One possibility is Hudson. It's written in Java, but there's integration with Python projects:</p>\n\n<blockquote>\n <p><a href=""http://redsolo.blogspot.com/2007/11/hudson-embraces-python.html"" rel=""noreferrer"">Hudson embraces Python</a></p>\n</blockquote>\n\n<p>I've never tried it myself, however.</p>\n\n<p>(<strong>Update</strong>, Sept. 2011: After a trademark dispute Hudson has been renamed to <a href=""http://jenkins-ci.org/"" rel=""noreferrer"">Jenkins</a>.)</p>",2013-01-28 03:54:17.217 UTC,28,"@dbr: Not totally true. The language something is written in often influences the software it works with best... build systems especially. When I looked at Hudson ~2 years ago it wanted unit test output only in JUnit's XML format; my Python unit test suite didn't do that. But if my project were in Java, it would have worked out of the box. In any case, it looks like integration has come a long way in both Hudson and Python's tools since then, and I am thankful for that.\nHudson being written in Java is as relevant as your the editor you use being written in C++! It's just a very well written, easy to setup and generally fully featured CI tool. Python can be very well integrated, `nosetests --with-xunit --with-coverage`, pylint with the ""Violations"" plugins, and ""Cobertura"" for coverage\nyep jenkins/hudson is perfectly fine, a breeze to set up."


In [256]:
deduped_df.shape

(683233, 5)

# Extract text from post body

In [261]:
%%time

content_list = []
url_list = []

for i, row in deduped_df.iterrows():
    content = row.body
    soup = BeautifulSoup(content, 'lxml')
    if soup.code: soup.code.decompose()
    tag_p = soup.p
    tag_pre = soup.pre
    text = ''
    if tag_p: text = text + tag_p.get_text()
    if tag_pre: text = text + tag_pre.get_text()
    if text =='': 
        continue
    tokens = tokenize_text(text)
    if len(tokens) <=3:
        continue
        
    content_list.append(' '.join(tokens))
    url_list.append('https://stackoverflow.com/a/' + str(row.id))


content_token_df = pd.DataFrame({'content_token': content_list, 'url': url_list})

CPU times: user 21min 36s, sys: 3.01 s, total: 21min 39s
Wall time: 21min 39s


In [262]:
content_token_df.head()

,content_token,url
0,"one possibility is hudson . it 's written in java , but there 's integration with python projects :",https://stackoverflow.com/a/538
1,can you show us your code ?,https://stackoverflow.com/a/783
2,the output of platform.system ( ) is as follows :,https://stackoverflow.com/a/1857
3,"dang -- lbrandy beat me to the punch , but that does n't mean i ca n't provide you with the system results for vista !",https://stackoverflow.com/a/1871
4,the list is dynamic and flexible but that flexibility comes at a speed cost .,https://stackoverflow.com/a/1987


In [263]:
content_token_df.shape

(644466, 2)

# Partition train/test/validate data

In [264]:
# train, valid, test splits
train, test = train_test_split(list(content_token_df.groupby('url')), train_size=0.87, shuffle=True, random_state=8081)
train, valid = train_test_split(train, train_size=0.82, random_state=8081)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [265]:
train = pd.concat([d for _, d in train]).reset_index(drop=True)
test = pd.concat([d for _, d in test]).reset_index(drop=True)
valid = pd.concat([d for _, d in valid]).reset_index(drop=True)

In [266]:
train.head()

,content_token,url
0,this is not a valid json string . use double quotes :,https://stackoverflow.com/a/6462436
1,"this is not a regular sentence where words are joined with an underscore . since you are just checking if the word is present , you may either remove ( as it is matching on a word boundary and _ is a word character ! ) or add alternatives : import re my_other_string = ' the_boat_has_sunk ' my_list = [ ' car ' , ' boat ' , ' truck ' ] my_list = re.compile(r'(?:\b|_)(?:%s)(?=\b|_ ) ' % ' |'.join(my_list ) ) if re.search(my_list , my_other_string ) : print('yay ' )",https://stackoverflow.com/a/30383962
2,"lists are zero indexed , so you need to set",https://stackoverflow.com/a/37092245
3,"in this case , a simple solution is to use dictionary comprehension :",https://stackoverflow.com/a/47307980
4,"it is likely your input file exceeds the soft memory limit in size . for big files use either or blobstoreziplineinputreader.def time_count_map(data ) : "" "" "" time count map function . "" "" "" text = data[1 ] try : reader = csv.reader([text.replace('\0 ' , ' ' ) ] , skipinitialspace = true ) for s in reader : "" "" "" calculate time elapsed "" "" "" sdw = s[1 ] start_date = time.strptime(sdw,""%m/%d/%y % i:%m:%s%p "" ) edw = s[2 ] end_date = time.strptime(edw,""%m/%d/%y % i:%m:%s%p "" ) time_difference = time.mktime(end_date ) - time.mktime(start_date ) yield ( s[0 ] , time_difference ) except indexerror , e : logging.debug(e )",https://stackoverflow.com/a/9256062


In [267]:
test.head()

,content_token,url
0,"first , create a list containing one hundred different colors . each color should be an rgb tuple , ranging from zero to one . there are many ways to do this , but i just typed in my favorite colors by hand .",https://stackoverflow.com/a/18622422
1,"i 've had a similar problem and i 've ended up adding a counter into the album equivalent . if the count is 0 and the operation is delete ( ) , then the album object is delete()d .",https://stackoverflow.com/a/5185748
2,two options that do n't require copying the whole set :,https://stackoverflow.com/a/59841
3,"i do n't think you can do this in one database query using django orm . but if it does n't bothers you , i would create a custom method on a model :",https://stackoverflow.com/a/2659296
4,for python 3 :,https://stackoverflow.com/a/26097790


In [268]:
valid.head()

,content_token,url
0,"if you read the docs you 'll see that the comment has an attribute ( unless it was deleted ) , so you should be able to do : response_text = ' good morning to you too , { } ! ' ... c.reply(response_text.format(c.author ) )",https://stackoverflow.com/a/39419165
1,"i think it would be better to use "" cls "" :",https://stackoverflow.com/a/28626798
2,the best python gui toolkit is wxpython ( also known as wxwidgets ) .,https://stackoverflow.com/a/4252766
3,"of course you could pass a list , but also you could just use * args . it sometimes depends on how you expect to use the function .",https://stackoverflow.com/a/32126746
4,yes and yes . you can get rid of the loops and it will speed up things :,https://stackoverflow.com/a/49798710


In [269]:
assert valid.shape[0] + test.shape[0] + train.shape[0] == content_token_df.shape[0],f'Row count mismatch.'

# Write to file

In [270]:
def write_to(df, filename, path='./data/stackoverflow/processed_data'):
    "Helper function to write processed files to disk."
    out = Path(path)
    out.mkdir(exist_ok=True)
    df.content_token.to_csv(out/'{}.content_token'.format(filename), index=False)
    df.url.to_csv(out/'{}.url'.format(filename), index=False)

In [271]:
write_to(train, 'train')
write_to(valid, 'valid')
write_to(test, 'test')

In [272]:
!ls -lah ./data/stackoverflow/processed_data

total 178M
drwxr-xr-x 4 root root 4.0K Nov 25 10:14 .
drwxr-xr-x 7 root root 4.0K Nov 24 16:33 ..
drwxr-xr-x 2 root root 4.0K Nov 25 10:14 .ipynb_checkpoints
drwxr-xr-x 2 root root 4.0K Nov 25 10:49 raw
-rw-r--r-- 1 root root  11M Nov 20 08:27 raw.csv
-rw-r--r-- 1 root root  11M Nov 20 08:25 raw9756.csv
-rw-r--r-- 1 root root  18M Nov 25 12:03 test.content_token
-rw-r--r-- 1 root root 3.0M Nov 25 12:03 test.url
-rw-r--r-- 1 root root  97M Nov 25 12:03 train.content_token
-rw-r--r-- 1 root root  17M Nov 25 12:03 train.url
-rw-r--r-- 1 root root  21M Nov 25 12:03 valid.content_token
-rw-r--r-- 1 root root 3.6M Nov 25 12:03 valid.url
